<h1>Product page parser</h1>

<p>In this notebook we are going to parse a single product page.</p>
<p>According to the task we should get the following: </p>
<ul>
  <li>categories</li>
  <li>subcategories</li> 
  <li>name</li>
  <li>price</li>
  <li>description</li>
  <li>charachteristics</li>
  <li>similar products</li>
  <li>product image link</li>
  <li>if product is a hit or not</li>
</ul> 

<h3><i>Important note</i></h3>
<p>Categories and subcategories (both level 1 and level 2) we plan to extract from parsing table obtained from the catalog main page (the parsing table currently contains 1222 rows. Each row has a link to products page with multiple product cards).</p>

<p>So from the product page we are going to extract the following:</p>
<ol>
  <li>name</li>
  <li>price</li>
  <li>description</li>
  <li>charachteristics</li>
  <li>similar products</li>
  <li>product image link</li>
  <li>if product is a hit or not</li>
</ol> 


In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup as BSoup
import pandas as pd
import re
import lxml
import html5lib

<p>We will develop our parser on a bosch drill webpage which seems to be the most complete and contains all the features, including hit tag.</p>

In [2]:
#url = 'https://www.oma.by/tros-dlya-rastyazhki-din3055-d1-5-mm-1-238091-p'
url = 'https://www.oma.by/perforator-bosch-pbh-2100-re-2-zubila-2-bura-1-216760-p'
single_product_page = requests.get(url)
product_page_soup = BSoup(single_product_page.text, 'html.parser')

product_page_special_soup = BSoup(single_product_page.text, 'html5lib')

<p style="color:green"><b>1. name </b></p>

In [3]:
product_name_raw = product_page_soup.select('div.page-title h1')
product_name = product_name_raw[0].text
print(product_name)

Перфоратор BOSCH PBH 2100 RE+2 зубила, 2 бура, Минск 


<p style="color:green"><b>2. price </b></p>

In [4]:
product_price_raw = product_page_soup.select('div.product-info-box_price')
product_price_text_raw = product_price_raw[0].text
product_price_text = product_price_text_raw.replace(" ", "")
product_price = str(product_price_text[:7])
print(product_price)


199,00


<p style="color:green"><b>3. description</b></p>

In [5]:
product_description_raw = product_page_soup.select('article.catalog-item-description-txt_content')
product_description = product_description_raw[0].text
print(product_description)


                                    BOSCH PBH 2100 RE универсальный и эргономичный перфоратор для бытового использования. Имеет 3 режима работы: сверление, долбление и бурение. Развивает скорость до 2300 об/мин. Патрон SDS-Plus позволяет быстро менять насадки. Оснащен функцией блокировки удара через кнопку выключения. Доступно завинчивание/вывинчивание шурупов через режим реверса. Дополнительная рукоятка обеспечивает равномерное распределение нагрузки. В комплекте предусмотрен ограничитель глубины для регулировки размера отверстий. Поставляется в чемодане с 2 зубилами и 2 бурами.                                


<p style="color:green"><b>4. characteristics </b></p>

In [6]:
product_characteristics_raw = product_page_soup.select('div.params-blocks')
if len(product_characteristics_raw) != 0:
    product_characteristics = product_characteristics_raw[0].text
    print(product_characteristics)
else:
    print('No parameters found')



Общие характеристики



Режим работы - сверление




Режим работы - сверление                                                                    
Возможность сверления.




Да



Режим работы - долбление

Да



Режимы работы - сверление с ударом

Да



Количество скоростей работы

1



Потребляемая мощность

550



Макс. число оборотов холостого хода

2300



Макс. количество ударов в минуту

5800



Макс. энергия удара

1.7



Питание

сеть




Технические характеристики



Шуруповерт

Да



Тип крепления бура

SDS-Plus



Макс. диаметр сверления (дерево)

30



Макс. диаметр сверления (металл)

13



Макс. диаметр сверления (бетон)

20




Функциональные особенности



Блокировка кнопки включения

Да



Фиксация шпинделя

Да



Электронная регулировка частоты вращения

Да




Комплектация



Реверс

Да



Дополнительная рукоятка

Да



Кейс в комплекте

Да



Ограничитель глубины сверления

Да



Комплектация

2 бура2 зубилаглубиномеринструкция по эксплуатациичемодан




Размеры и 

<p style="color:red"><b>5. similar products </b></p>
<p>Failed to extract similar products.</p>

In [7]:
similar_products_raw = product_page_special_soup.find(id = "sales_shared")
#similar_products_raw = product_page_soup.findAll('div',{'class':'sales_shared'})
#similar > div
#product_name = product_name_raw[0].text
print(similar_products_raw)


None


<p style="color:green"><b>6. product image link </b></p>

In [8]:
product_image_link_raw = product_page_soup.select('div.slider-w-preview img')
product_image_link = 'https://www.oma.by' + product_image_link_raw[0].get('src')
print(product_image_link)

https://www.oma.by/upload/Sh/imageCache/03a/cc1/1.216760-medium.jpg


<p style="color:green"><b>7. if product is a hit or not </b></p>

In [9]:
hit_offer_raw = product_page_soup.findAll('span',{'class':'icon special-icon special-icon__hit product-item_special'})
print(hit_offer_raw)
if len(hit_offer_raw) != 0:
    product_is_hit = True
    print('\nThe product is hit!')
else:
    product_is_hit = False
    print('\nThe product is not hit.')

[<span class="icon special-icon special-icon__hit product-item_special"></span>]

The product is hit!


<h3>Create product table</h3>

In [10]:
product_df = pd.DataFrame(columns = ['name',\
                                     'price','description', 'characteristics', \
                                     'product image link', 'is_hit']) 
product_df

,name,price,description,characteristics,product image link,is_hit


In [11]:
product_df.loc[0] = [product_name] + [product_price]  \
                  + [product_description] + [product_characteristics] \
                  + [product_image_link] + [product_is_hit]
product_df.head()                       

,name,price,description,characteristics,product image link,is_hit
0,"Перфоратор BOSCH PBH 2100 RE+2 зубила, 2 бура,...","\n199,00",\n BOSCH PB...,\n\nОбщие характеристики\n\n\n\nРежим работы -...,https://www.oma.by/upload/Sh/imageCache/03a/cc...,True


<h3>Product parameters function</h3>
<p>Construct function that returns a dictionary with product parameters. That is sketchy version of a function. Later will be converted into normal view.</p> 

In [12]:
def get_product_parameters(soup):
    
    #product name
    product_name_raw = soup.select('div.page-title h1')
    product_name = product_name_raw[0].text
    
    #product_price
    product_price_raw = soup.select('div.product-info-box_price')
    product_price_text_raw = product_price_raw[0].text
    product_price_text = product_price_text_raw.replace(" ", "")
    product_price = str(product_price_text[:7])
    
    #product_description
    product_description_raw = soup.select('article.catalog-item-description-txt_content')
    product_description = product_description_raw[0].text
    
    #product_characteristics
    product_characteristics_raw = soup.select('div.params-blocks')
    if len(product_characteristics_raw) != 0:
        product_characteristics = product_characteristics_raw[0].text
    else:
        product_characteristics = ''
        
    #similar_products
    similar_products = ''
        
    #product_image_link    
    product_image_link_raw = soup.select('div.slider-w-preview img')
    product_image_link = 'https://www.oma.by' + product_image_link_raw[0].get('src')
    
    #product_is_hit
    hit_offer_raw = soup.findAll('span',{'class':'icon special-icon special-icon__hit product-item_special'})
    if len(hit_offer_raw) != 0:
        product_is_hit = True

    else:
        product_is_hit = False

                                                      
    dict = {
        'product_name' : product_name,
        'product_price' : product_price,
        'product_description' : product_description,
        'product_characteristics' : product_characteristics,
        'similar_products' : similar_products,
        'product_image_link' : product_image_link,
        'product_is_hit' : product_is_hit,
    }
    
    
    return dict
    

In [13]:
product_parameters_dict = get_product_parameters(product_page_soup)
for param in product_parameters_dict:
    print(f'\n\n{param} {product_parameters_dict[param]}')



product_name Перфоратор BOSCH PBH 2100 RE+2 зубила, 2 бура, Минск 


product_price 
199,00


product_description 
                                    BOSCH PBH 2100 RE универсальный и эргономичный перфоратор для бытового использования. Имеет 3 режима работы: сверление, долбление и бурение. Развивает скорость до 2300 об/мин. Патрон SDS-Plus позволяет быстро менять насадки. Оснащен функцией блокировки удара через кнопку выключения. Доступно завинчивание/вывинчивание шурупов через режим реверса. Дополнительная рукоятка обеспечивает равномерное распределение нагрузки. В комплекте предусмотрен ограничитель глубины для регулировки размера отверстий. Поставляется в чемодане с 2 зубилами и 2 бурами.                                


product_characteristics 

Общие характеристики



Режим работы - сверление




Режим работы - сверление                                                                    
Возможность сверления.




Да



Режим работы - долбление

Да



Режимы работы - сверление 

In [14]:
from parsing_module import *

In [15]:
url = 'https://www.oma.by/gvozdi-3-0kh80-mm-1-209150-p'
content = simple_get(url)
soup = get_soup(content)
product_dict = get_product_parameters(soup)
for x in product_dict:
    print(f'\n{x}: {product_dict[x]}')



product_name: Гвозди 3,0х80 мм, Минск 

product_price: 
3,02/к

product_description: 
                                    Гвозди предназначены для строительных работ и крепления деревянных элементов между собой. Используются для сборки мебели, ремонтных работ в доме и на участке. Гвоздь отличается конусным подголовником, который при забивании расклинивается в отверстии. Благодаря этому в пять раз увеличивается прочность крепления элементов. Плоская, потайная головка, позволяет замаскировать место соединения. Изделие выполнено из металла, который обеспечивает высокую степень защиты крепежных элементов. Размер — 3х80 мм. В 1 кг примерно 204 гвоздя.

product_characteristics: 

Основные характеристики



Назначение

строительный



Материал

сталь




Размеры и вес



Длина

80



Диаметр стержня

3







                                    Все характеристики

similar_products: 

product_image_link: https://www.oma.by/upload/Sh/imageCache/80f/f48/1.209150-medium.jpg

product_is_hit: Fals

In [24]:
url = 'https://www.oma.by/drel-shurupovert-akkumulyatornaya-bosch-easydrill-12-2-060397290x-1-247573-p'
content = simple_get(url)
soup = get_soup(content)

In [25]:
#similar_products = soup.select('class.sales_shared')
#similar_products = soup.findAll('span',{'class':'icon special-icon special-icon__hit product-item_special'})
x = soup.find("div", {"id": "comp_fc90478f4a22d1dc40c96e34ff844bfa"})
print(x)
print(similar_products)

<div id="comp_fc90478f4a22d1dc40c96e34ff844bfa"><a class="js_smart_cache_load" href="/drel-shurupovert-akkumulyatornaya-bosch-easydrill-12-2-060397290x-1-247573-p?SMARTCAHEID=eJztk01v2zAMhv%2BLzkbhDzWxc1NtpVEjS4KtDDW6wUiT%2BJRtwNAchqL67SNVZ0njpOvO20WQXlIUSfF5JqwojDYLySoyefqx2wSE3bH7ttQFJxPSkP6sjRVateAtcMMkGE9sM1FbXTVgUCeWu0Vpzsi1bSTv9ZzlM95aUaKQjMIwPIiNQZGhoEujFVe2tbw0klk0XK033XK3ffL2hbJaGXYLhjgMyFRUtW2lZkX%2FTj0XBnOAwPl8r5Wssv6tvuKDAEUVvK14vZD2uAKB8bpVFtJx2tFlHK%2Bj9YqGq2y0SWjXpZQ%2Bdktw5pKXmK73p1kaX2cYX5QCGt6aSpuaTJ5JlCVgxzUgUZrhHtaXgGDtr5ev8R7PfeO8ECUpHYMobqTO514bBaTit%2BCBMbXC7%2BBtDTnkkHwUkEKXTICRfP%2B6vHr8CZfhKjuETOIkoxjy%2BKCY%2F5PPu5BGK1yT1K9rXGnk90ufnJy29cxX9EASTC0hX%2FYZHSxxhuqpY7DXzUwo6K8q%2BD0Widngvq8ZjoY1rxPxAO4Fl%2BITx6HDU76oKq5yHMGbxk%2BVn%2BJvu%2B02IK9t8lkaXgkN8cK92ue412O4KSyGicZRiD%2BGx%2FajnTCVyH%2BPLbaQ5f28w39gZ7FkMxXScpyBI%2BdonIywF747%2Fc%2FxYtAmGAw35NadguveI9ddRtddYNddgtedpdcN8HV%2F4tcNAHYDgt05hN2AYfcuxO6vKHbnMXYf59i9BdmdJdm9Rdn9Z%2FlfYPnlFzde8iI%3D" onclick="BX.a

In [ ]:
<div id="comp_fc90478f4a22d1dc40c96e34ff844bfa" class="" style="opacity: 1;"><script type="text/javascript" bxrunfirst="true">
var arAjaxPageData = {'TITLE':'Каталог товаров - Компания ОМА – все для строительства и ремонта, (Минск)','WINDOW_TITLE':'Каталог товаров - Компания ОМА – все для строительства и ремонта, (Минск)','SCRIPTS':['/bitrix/js/main/core/core_window.js?152841120497284','/bitrix/js/socialservices/ss_admin.js?14704178962117'],'NAV_CHAIN':'\n    <nav class=\"breadcrumbs\">\n                                                <div  itemscope=\"\" itemtype=\"http://data-vocabulary.org/Breadcrumb\"  class=\"breadcrumbs_item breadcrumbs_item__first \">\n                    						<a href=\"/\" itemprop=\"url\" class=\"breadcrumbs_link\"><span itemprop=\"title\">Главная<\/span><\/a>\n                                    <\/div>\n            \n            <div class=\"breadcrumbs_separator\"><\/div>\n                                        <div class=\"breadcrumbs_item breadcrumbs_item__back\">\n                    <span class=\"icon icon__gray-chevron-left\"><\/span>\n                    <span class=\"icon icon__chevron-left icon__reflex\"><\/span>\n                    <a href=\"/\" class=\"breadcrumbs_link\">Главная<\/a>\n                <\/div>\n            \n            \n\n        \n    <\/nav>\n   \n\n'};
parent.BX.ajax.UpdatePageData(arAjaxPageData);
</script><script type="text/javascript"></script>








    <script>
        var eccomerce_json_catalog_bottom_ajax={"ecommerce":{"currencyCode":"USD","impressions":[{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u0431\u044b\u0442\u043e\u0432\u043e\u0439 WERKER EWRH 606","id":"1.229957","price":53.41,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Watt Pro WBH-650","id":"1.224343","price":73.39,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Watt WBH-850","id":"1.178674","price":72.58,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u043f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u044c\u043d\u044b\u0439 ANCHOR Z1C-TM05-22","id":"1.233091","price":50.77,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 3000-2 FRE","id":"1.156915","price":181.22,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Einhell BT-RH 920 E","id":"1.225016","price":86.1,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u0431\u044b\u0442\u043e\u0432\u043e\u0439 BOSCH PBH 2000 RE, 06033A9322","id":"1.257514","price":87.27,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2100 RE","id":"1.190165","price":95.38,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 3000 FRE","id":"1.190166","price":161.67,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH262","id":"1.236694","price":72.24,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2500 RE","id":"1.201711","price":112.07,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2900 RE","id":"1.201629","price":134.48,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 WERKER EWRH650, 800 \u0412\u0442","id":"1.229958","price":81.07,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2100 RE","id":"1.216760","price":94.9,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Einhell BT-RH 900\/1","id":"1.191621","price":132.37,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u0431\u044b\u0442\u043e\u0432\u043e\u0439 PATRIOT RH265Q","id":"1.238780","price":88.8,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 DWT SBH 06-20 T BMC","id":"1.220202","price":87.12,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH260","id":"1.241138","price":76.24,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH232","id":"1.241137","price":58,"list":"related"},{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH240","id":"1.236695","price":69.39,"list":"related"}]},"event":"EEC impression"};

        $( document ).ready(function() {
            if(!$("body").hasClass("js-related-yes")) {
                $("body").addClass("js-related-yes");

                window.dataLayer = window.dataLayer || [];
                dataLayer.push(eccomerce_json_catalog_bottom_ajax);

            }
        });

    </script>

    <script data-skip-moving="true">
        var eccomerce_json_catalog_bottom={"488748":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u0431\u044b\u0442\u043e\u0432\u043e\u0439 WERKER EWRH 606","id":"1.229957","price":53.41,"quantity":1},"274704":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Watt Pro WBH-650","id":"1.224343","price":73.39,"quantity":1},"264035":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Watt WBH-850","id":"1.178674","price":72.58,"quantity":1},"489830":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u043f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u044c\u043d\u044b\u0439 ANCHOR Z1C-TM05-22","id":"1.233091","price":50.77,"quantity":1},"261490":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 3000-2 FRE","id":"1.156915","price":181.22,"quantity":1},"275028":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Einhell BT-RH 920 E","id":"1.225016","price":86.1,"quantity":1},"549920":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u0431\u044b\u0442\u043e\u0432\u043e\u0439 BOSCH PBH 2000 RE, 06033A9322","id":"1.257514","price":87.27,"quantity":1},"265383":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2100 RE","id":"1.190165","price":95.38,"quantity":1},"265384":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 3000 FRE","id":"1.190166","price":161.67,"quantity":1},"436580":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH262","id":"1.236694","price":72.24,"quantity":1},"267930":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2500 RE","id":"1.201711","price":112.07,"quantity":1},"351887":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2900 RE","id":"1.201629","price":134.48,"quantity":1},"484911":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 WERKER EWRH650, 800 \u0412\u0442","id":"1.229958","price":81.07,"quantity":1},"271493":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2100 RE","id":"1.216760","price":94.9,"quantity":1},"265743":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Einhell BT-RH 900\/1","id":"1.191621","price":132.37,"quantity":1},"481898":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u0431\u044b\u0442\u043e\u0432\u043e\u0439 PATRIOT RH265Q","id":"1.238780","price":88.8,"quantity":1},"272768":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 DWT SBH 06-20 T BMC","id":"1.220202","price":87.12,"quantity":1},"481909":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH260","id":"1.241138","price":76.24,"quantity":1},"475962":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH232","id":"1.241137","price":58,"quantity":1},"436581":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH240","id":"1.236695","price":69.39,"quantity":1}};
        var eccomerce_json_catalog_no_qnt_bottom={"488748":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u0431\u044b\u0442\u043e\u0432\u043e\u0439 WERKER EWRH 606","id":"1.229957","price":53.41},"274704":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Watt Pro WBH-650","id":"1.224343","price":73.39},"264035":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Watt WBH-850","id":"1.178674","price":72.58},"489830":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u043f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u044c\u043d\u044b\u0439 ANCHOR Z1C-TM05-22","id":"1.233091","price":50.77},"261490":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 3000-2 FRE","id":"1.156915","price":181.22},"275028":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Einhell BT-RH 920 E","id":"1.225016","price":86.1},"549920":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u0431\u044b\u0442\u043e\u0432\u043e\u0439 BOSCH PBH 2000 RE, 06033A9322","id":"1.257514","price":87.27},"265383":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2100 RE","id":"1.190165","price":95.38},"265384":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 3000 FRE","id":"1.190166","price":161.67},"436580":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH262","id":"1.236694","price":72.24},"267930":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2500 RE","id":"1.201711","price":112.07},"351887":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2900 RE","id":"1.201629","price":134.48},"484911":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 WERKER EWRH650, 800 \u0412\u0442","id":"1.229958","price":81.07},"271493":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Bosch PBH 2100 RE","id":"1.216760","price":94.9},"265743":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 Einhell BT-RH 900\/1","id":"1.191621","price":132.37},"481898":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 \u0431\u044b\u0442\u043e\u0432\u043e\u0439 PATRIOT RH265Q","id":"1.238780","price":88.8},"272768":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 DWT SBH 06-20 T BMC","id":"1.220202","price":87.12},"481909":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH260","id":"1.241138","price":76.24},"475962":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH232","id":"1.241137","price":58},"436581":{"name":"\u041f\u0435\u0440\u0444\u043e\u0440\u0430\u0442\u043e\u0440 PATRIOT RH240","id":"1.236695","price":69.39}};

        function addBasketGTM_bottom(list,productID){
            var product=eccomerce_json_catalog_bottom[productID];

            window.dataLayer = window.dataLayer || [];
            dataLayer.push({
                "event": "addToCart",
                "ecommerce": {
                    "currencyCode": "USD",
                    "add": {
                        "actionField": {
                            "list": list
                        },
                        "products": [product]
                    }
                }
            });
        }


        function productClickGTM_bottom(list,productID,link){
            var product=eccomerce_json_catalog_no_qnt_bottom[productID];

            window.dataLayer = window.dataLayer || [];
            dataLayer.push({
                "event": "productClick",
                'ecommerce': {
                    'currencyCode': 'USD',
                    'click': {
                        'actionField': {'list': list},
                        'products': [product]
                    }
                },
                'eventCallback': function () {
                    document.location = link;
                }

            });
        }

        function addMoreJTM_bottom(list,productID){
            var product=eccomerce_json_catalog_no_qnt_bottom[productID];

            window.dataLayer = window.dataLayer || [];
            dataLayer.push({
                "event": "productClick",
                'ecommerce': {
                    'currencyCode': 'USD',
                    'click': {
                        'actionField': {'list': list},
                        'products': [product]
                    }
                }
            });
        }

        function addQuickJTM_bottom(list,productID){
            var product=eccomerce_json_catalog_no_qnt_bottom[productID];

            window.dataLayer = window.dataLayer || [];
            dataLayer.push({
                "event":"quickView",
                "ecommerce": {
                    "detail": {
                        "actionField": {
                            "list": list
                        },
                        "products": [product]
                    },
                }
            });

            window.dataLayer = window.dataLayer || [];
            dataLayer.push({
                "event": "productClick",
                'ecommerce': {
                    'currencyCode': 'USD',
                    'click': {
                        'actionField': {'list': list},
                        'products': [product]
                    }
                }
            });

        }

    </script>

























<section style="margin-top:30px;" class="catalog-item_section catalog-item-product-section js-accordion js-accordion-sm open catalog-item_section__stacked js_similar " id="similar">
    <header class="catalog-item_section-header js-accordion-bar open">
        <span class="icon icon__gray-chevron-down"></span>
        <h2>Похожие товары</h2>
    </header>

    <div class="catalog-item_section-body catalog-item_section-slider js-accordion-body">
        <div class="slider product_slider js-to-max-height-wrap js-product-slider js_catalog_list slick-initialized slick-slider" style="height: auto; overflow: visible;"><button type="button" data-role="none" class="slick-prev slick-arrow slick-disabled" aria-label="Previous" role="button" aria-disabled="true" style="display: inline-block;">Previous</button>

                            


                            

                            


                            

                            


                            

                            


                            


                            


                            


                            


                            


                            


                            


                            

                            

                            

                            

                            

                            


            

        <div aria-live="polite" class="slick-list draggable"><div class="slick-track" style="opacity: 1; width: 3384px; transform: translate3d(0px, 0px, 0px);" role="listbox"><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item488748 slick-slide slick-current slick-active" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=488748&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="0" role="option" aria-describedby="slick-slide40" data-slick-index="0" aria-hidden="false">

    <div class="product-item_img-box">
        <img alt="Перфоратор бытовой WERKER EWRH 606" class="" width="160" height="150" title="Перфоратор бытовой WERKER EWRH 606" data-src="/upload/Sh/imageCache/740/609/1.229957-small.JPG" src="/upload/Sh/imageCache/740/609/1.229957-small.JPG">

        
        
                                                <span style="z-index:5;" class="icon special-icon special-icon__red product-item_special">-20%</span>
            </div>
    <div class="product-item_title ABtitle">Перфоратор бытовой WERKER EWRH 606</div>

    
    <div class="product-item_price price-block">
                    <span class="price-block_old">
                <span class="price__old">139,<small>99</small></span>
                <span class="price-block_save price__save">Экономия 28,<small>00</small></span>
            </span>
        

        <span class="price__normal">
                            111,<small>99</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="0">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="0">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=488748" tabindex="0">
                        <span class="icon icon__small-fav-list wishlist_product-488748"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-bytovoy-werker-ewrh-606-1-229957-p" tabindex="0" class="area-link"></a>
    
    
</div><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item264035 slick-slide slick-active" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=264035&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="0" role="option" aria-describedby="slick-slide41" data-slick-index="1" aria-hidden="false">

    <div class="product-item_img-box">
        <img alt="Перфоратор Watt WBH-850" class="" width="160" height="150" title="Перфоратор Watt WBH-850" data-src="/upload/Sh/imageCache/0cb/830/1.178674-small.jpg" src="/upload/Sh/imageCache/0cb/830/1.178674-small.jpg">

        
        
                            </div>
    <div class="product-item_title ABtitle">Перфоратор Watt WBH-850</div>

    
    <div class="product-item_price price-block">
        

        <span class="price__normal">
                            152,<small>20</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="0">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="0">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=264035" tabindex="0">
                        <span class="icon icon__small-fav-list wishlist_product-264035"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-watt-wbh-850-5-850-026-00-1-178674-p" tabindex="0" class="area-link"></a>
    
    
</div><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item261490 slick-slide slick-active" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=261490&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="0" role="option" aria-describedby="slick-slide42" data-slick-index="2" aria-hidden="false">

    <div class="product-item_img-box">
        <img alt="Перфоратор Bosch PBH 3000-2 FRE" class="" width="160" height="150" title="Перфоратор Bosch PBH 3000-2 FRE" data-src="/upload/Sh/imageCache/250/00f/1.156915-small.jpg" src="/upload/Sh/imageCache/250/00f/1.156915-small.jpg">

        
        
                            </div>
    <div class="product-item_title ABtitle">Перфоратор Bosch PBH 3000-2 FRE</div>

    
    <div class="product-item_price price-block">
        

        <span class="price__normal">
                            379,<small>99</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="0">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="0">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=261490" tabindex="0">
                        <span class="icon icon__small-fav-list wishlist_product-261490"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-bosch-pbh-3000-2-fre-1-156915-p" tabindex="0" class="area-link"></a>
    
    
</div><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item549920 slick-slide" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=549920&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="-1" role="option" aria-describedby="slick-slide43" data-slick-index="3" aria-hidden="true">

    <div class="product-item_img-box">
        <img alt="Перфоратор бытовой BOSCH PBH 2000 RE, 06033A9322" class="" width="160" height="150" title="Перфоратор бытовой BOSCH PBH 2000 RE, 06033A9322" data-src="/upload/Sh/imageCache/253/a3c/1.257514-small.jpg" src="/upload/Sh/imageCache/253/a3c/1.257514-small.jpg">

        
        
                            </div>
    <div class="product-item_title ABtitle">Перфоратор бытовой BOSCH PBH 2000 RE, 06033A9322</div>

    
    <div class="product-item_price price-block">
        

        <span class="price__normal">
                            183,<small>00</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="-1">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="-1">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=549920" tabindex="-1">
                        <span class="icon icon__small-fav-list wishlist_product-549920"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-bytovoy-bosch-pbh-2000-re-06033a9322-1-257514-p" tabindex="-1" class="area-link"></a>
    
    
</div><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item265383 slick-slide" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=265383&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="-1" role="option" aria-describedby="slick-slide44" data-slick-index="4" aria-hidden="true">

    <div class="product-item_img-box">
        <img alt="Перфоратор Bosch PBH 2100 RE" class="" width="160" height="150" title="Перфоратор Bosch PBH 2100 RE" data-src="/upload/Sh/imageCache/f70/69b/1.190165-small.jpg" src="/upload/Sh/imageCache/f70/69b/1.190165-small.jpg">

        
        
                            </div>
    <div class="product-item_title ABtitle">Перфоратор Bosch PBH 2100 RE</div>

    
    <div class="product-item_price price-block">
        

        <span class="price__normal">
                            200,<small>00</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="-1">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="-1">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=265383" tabindex="-1">
                        <span class="icon icon__small-fav-list wishlist_product-265383"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-bosch-pbh-2100-re-1-190165-p" onclick="BX.ajax.insertToNode('/perforator-bosch-pbh-2100-re-1-190165-p?bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa', 'comp_fc90478f4a22d1dc40c96e34ff844bfa'); return false;" tabindex="-1" class="area-link"></a>
    
    
</div><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item265384 slick-slide" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=265384&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="-1" role="option" aria-describedby="slick-slide45" data-slick-index="5" aria-hidden="true">

    <div class="product-item_img-box">
        <img alt="Перфоратор Bosch PBH 3000 FRE" class="" width="160" height="150" title="Перфоратор Bosch PBH 3000 FRE" data-src="/upload/Sh/imageCache/10c/93c/1.190166-small.jpg" src="/upload/Sh/imageCache/10c/93c/1.190166-small.jpg">

        
        
                            </div>
    <div class="product-item_title ABtitle">Перфоратор Bosch PBH 3000 FRE</div>

    
    <div class="product-item_price price-block">
        

        <span class="price__normal">
                            339,<small>00</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="-1">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="-1">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=265384" tabindex="-1">
                        <span class="icon icon__small-fav-list wishlist_product-265384"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-bosch-pbh-3000-fre-0603393220-1-190166-p" tabindex="-1" class="area-link"></a>
    
    
</div><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item436580 slick-slide" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=436580&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="-1" role="option" aria-describedby="slick-slide46" data-slick-index="6" aria-hidden="true">

    <div class="product-item_img-box">
        <img alt="Перфоратор PATRIOT RH262" class="" width="160" height="150" title="Перфоратор PATRIOT RH262" data-src="/upload/Sh/imageCache/013/24e/1.236694-small.jpg" src="/upload/Sh/imageCache/013/24e/1.236694-small.jpg">

        
        
                            </div>
    <div class="product-item_title ABtitle">Перфоратор PATRIOT RH262</div>

    
    <div class="product-item_price price-block">
        

        <span class="price__normal">
                            151,<small>47</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="-1">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="-1">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=436580" tabindex="-1">
                        <span class="icon icon__small-fav-list wishlist_product-436580"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-patriot-rh262-1-236694-p" tabindex="-1" class="area-link"></a>
    
    
</div><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item267930 slick-slide" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=267930&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="-1" role="option" aria-describedby="slick-slide47" data-slick-index="7" aria-hidden="true">

    <div class="product-item_img-box">
        <img alt="Перфоратор Bosch PBH 2500 RE" class="" width="160" height="150" title="Перфоратор Bosch PBH 2500 RE" data-src="/upload/Sh/imageCache/ac9/5f2/1.201711-small.jpg" src="/upload/Sh/imageCache/ac9/5f2/1.201711-small.jpg">

        
        
                            </div>
    <div class="product-item_title ABtitle">Перфоратор Bosch PBH 2500 RE</div>

    
    <div class="product-item_price price-block">
        

        <span class="price__normal">
                            235,<small>00</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="-1">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="-1">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=267930" tabindex="-1">
                        <span class="icon icon__small-fav-list wishlist_product-267930"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-bytovoy-bosch-pbh-2500-re-600-vt-1-201711-p" tabindex="-1" class="area-link"></a>
    
    
</div><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item351887 slick-slide" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=351887&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="-1" role="option" aria-describedby="slick-slide48" data-slick-index="8" aria-hidden="true">

    <div class="product-item_img-box">
        <img alt="Перфоратор Bosch PBH 2900 RE" class="" width="160" height="150" title="Перфоратор Bosch PBH 2900 RE" data-src="/upload/Sh/imageCache/14f/3ea/1.201629-small.jpg" src="/upload/Sh/imageCache/14f/3ea/1.201629-small.jpg">

        
        
                            </div>
    <div class="product-item_title ABtitle">Перфоратор Bosch PBH 2900 RE</div>

    
    <div class="product-item_price price-block">
        

        <span class="price__normal">
                            282,<small>00</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="-1">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="-1">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=351887" tabindex="-1">
                        <span class="icon icon__small-fav-list wishlist_product-351887"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-bosch-pbh-2900-re-1-201629-p" tabindex="-1" class="area-link"></a>
    
    
</div><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item484911 slick-slide" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=484911&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="-1" role="option" aria-describedby="slick-slide49" data-slick-index="9" aria-hidden="true">

    <div class="product-item_img-box">
        <img alt="Перфоратор WERKER EWRH650, 800 Вт" class="" width="160" height="150" title="Перфоратор WERKER EWRH650, 800 Вт" data-src="/upload/Sh/imageCache/0fd/543/1.229958-small.jpg" src="/upload/Sh/imageCache/0fd/543/1.229958-small.jpg">

        
        
                            </div>
    <div class="product-item_title ABtitle">Перфоратор WERKER EWRH650, 800 Вт</div>

    
    <div class="product-item_price price-block">
        

        <span class="price__normal">
                            169,<small>99</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="-1">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="-1">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=484911" tabindex="-1">
                        <span class="icon icon__small-fav-list wishlist_product-484911"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-werker-ewrh650-800-vt-1-229958-p" tabindex="-1" class="area-link"></a>
    
    
</div><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item271493 slick-slide" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=271493&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="-1" role="option" aria-describedby="slick-slide410" data-slick-index="10" aria-hidden="true">

    <div class="product-item_img-box">
        <img alt="Перфоратор Bosch PBH 2100 RE" class="" width="160" height="150" title="Перфоратор Bosch PBH 2100 RE" data-src="/upload/Sh/imageCache/fce/232/1.216760-small.jpg" src="/upload/Sh/imageCache/fce/232/1.216760-small.jpg">

        
        
                            </div>
    <div class="product-item_title ABtitle">Перфоратор Bosch PBH 2100 RE</div>

    
    <div class="product-item_price price-block">
        

        <span class="price__normal">
                            199,<small>00</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="-1">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="-1">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=271493" tabindex="-1">
                        <span class="icon icon__small-fav-list wishlist_product-271493"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-bosch-pbh-2100-re-2-zubila-2-bura-1-216760-p" tabindex="-1" class="area-link"></a>
    
    
</div><div class="sales_shared slider-section_slide product-item js-to-max-height js_cat_list_item   js_cat_list_item436581 slick-slide" rel="/perforator-bosch-pbh-2100-re-1-190165-p?BASKET_ADD=436581&amp;COUNT=1&amp;SMARTCAHEID=eJztU01v2zAM%2FS86G4Vtuc7HTbWVRo0sCbIy1NgKI03iU7YBQ3MYiuq3T1TsJo2drj1vF0F8pCjyke8ZkTxXUi050Wj69Gu%2FDRC5I%2Fd1IXOKpqhCrS2VYVLULprBhXDnPPPNWWmkrpxDnHnuloUagEtTcdriGcnmtDasAACnYRgewUoBSACQhZKCClMbWihODDiuNttmtd89ef9SGCkUuXWOOAzQjOnS1FySvP2nXDAFNbjE2aLDCqKN%2F6vt%2BAi4pnJaa1ouuTntgEG%2BZj0Jk9G4SVZxvIk26yRcT9ItTppmnCSPzcoFU04LKNfHx%2Bk1HmPIzwrmCK%2BVlqpE068PAYImD1EpBNDMM%2BSBCI%2BvEweyGy6zhcfSAGl66yLQ9BlJAbzTunSfZaabYi4Lwpwf%2Ffy%2Bunr87d671%2BSYFcd44rOeGoJ4%2Fr%2FtwyRaw4nH%2FtzAmUT%2BvvL18Vldzg%2FVIzxyEEYPXVFHTzwB9Dww6HA1Z8JxKXJ6D31CNXBv23amItVh%2BkBRTjn7QmHBwMqWWlORwbrdVH6D%2FMb%2B2O92ATow5atUVDPp8oUd2tbY4bF7yQykiUZROEEHs%2F4oE0qz7HVFgUKStbvtRgLMQstqxrihMO%2BT4GiEU78MwE47PJr3aHoJkO1r1J6L1L6nUntZpvaCTu0lodpBpdqeVO3ftGp7YrU9tdohudqeXu27grWfUqwdlqwd0Kx9K1o7qFr7Vrb2v27%2FEd2%2B%2FAFfw%2BxO&amp;bxajaxid=fc90478f4a22d1dc40c96e34ff844bfa&amp;ELEMENT_PRECODE=perforator-bosch-pbh-2100-re-1-&amp;ELEMENT_CODE=1.190165" style="width: 282px; height: 342px;" tabindex="-1" role="option" aria-describedby="slick-slide411" data-slick-index="11" aria-hidden="true">

    <div class="product-item_img-box">
        <img alt="Перфоратор PATRIOT RH240" class="" width="160" height="150" title="Перфоратор PATRIOT RH240" data-src="/upload/Sh/imageCache/d10/439/1.236695-small.jpg" src="/upload/Sh/imageCache/d10/439/1.236695-small.jpg">

        
        
                            </div>
    <div class="product-item_title ABtitle">Перфоратор PATRIOT RH240</div>

    
    <div class="product-item_price price-block">
        

        <span class="price__normal">
                            145,<small>51</small>
                
                    </span>

    </div>

            <div class="product-item_bottom ABbtn">
            <div class="product-item_btn-box">
                                                            <button type="button" class="btn btn__blue btn__block js_orderButton showNotInOrder " tabindex="-1">
                            <span class="btn_icon icon icon__small-basket"></span>В корзину
                        </button>
                    
                    <a href="/checkout/?STEP=2" class=" btn btn__orange btn__block showInOrder hide" tabindex="-1">Оформить</a>
                            </div>

			            <div class="product-item_star-bucks wish-list-block-hide">
                <div class="wishlist-top product-item_star-bucks-item icon-reflex-parent tooltip-parent tooltip-btn">
                    <a href="#" class="js-open-popup" data-popup-url="/local/ajax/wishlist/list.php?product_id=436581" tabindex="-1">
                        <span class="icon icon__small-fav-list wishlist_product-436581"></span>
                        <span class="icon icon__small-fav-list icon__reflex"></span>
                    </a>
                </div>
            </div>
			

        </div>
    
            <a href="/perforator-patriot-rh240-1-236695-p" tabindex="-1" class="area-link"></a>
    
    
</div></div></div><button type="button" data-role="none" class="slick-next slick-arrow" aria-label="Next" role="button" style="display: block;" aria-disabled="false">Next</button></div>
    </div>
</section>
</div>